# Компьютерное зрение. Базовый курс

## Алгоритмы восстановления изображений из CFA (demosaicing algorithms)

### Алгоритм Variable Number of Gradients (VNG)

#### Условие

**Дано:** исходное изображение (оригинал, reference) и соответствующее ему полутоновое изображение матрицы Байера.

**Необходимо:**

* реализовать один из алгоритмов demosaicing-а (в данной работе выбран алгоритм `VNG`);

* восстановить с его помощью цветное изображение из полутонового;

* проанализировать полученное изображение и сделать вывод, во сколько раз снижается разрешение восстановленного изображения по сравнению с оригиналом;

* вычислить метрику `PSNR` между оригиналом и полученным изображением (для яркости);

* оценить время работы алгоритма (сек/мегапиксель).

#### Решение

Для работы с изображениями (методы, необходимые для чтения изображения из файла, записи изображения в файл и отображения изображения на экране) воспользуемся библиотекой `OpenCV`, а для более быстрой и эффективной работы с массивами - библиотекой `numpy`.

In [1]:
import cv2
import numpy as np

Загрузим и сохраним полутоновое и оригинальное изображения.

In [2]:
rgb_cfa = cv2.imread('RGB_CFA.bmp', cv2.IMREAD_COLOR)
rgb_cfa.shape

(2073, 4176, 3)

In [3]:
def viewImage(image, name_of_window):
    cv2.namedWindow(name_of_window, cv2.WINDOW_NORMAL)
    cv2.imshow(name_of_window, image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [4]:
viewImage(rgb_cfa,'RGB_CFA.bmp')

In [5]:
orig_img = cv2.imread('Original.bmp', cv2.IMREAD_COLOR)
orig_img.shape

(2073, 4176, 3)

In [6]:
viewImage(orig_img,'Original.bmp')

Теперь можно приступать к реализации самого алгоритма.

"For each pixel, we recover the missing color components as follows.

First, a set of gradients is determined from the color values in the $5 \times 5$ neighborhood centered at the pixel under consideration. Each gradient corresponds to a different direction.

Second, for each set of gradients, a threshold value is determined, and the threshold is used to select a subset of gradients. Low-valued gradients indicate pixels having similar color values whereas high-valued gradients would be expected in regions of the image where there are many fine details or sharp edges.

Third, the subset of gradients is used to locate regions of pixels that are most like the pixel under consideration. Note that the pixels in the identified regions can lie in more than one direction from the pixel under consideration, in contrast to previous work where color values located in only a single direction are used for interpolation.

The pixels in the regions are then weighted and summed to determine the average difference between the color of the actual measured center pixel value and the missing color."

In [7]:
k1 = 1.5
k2 = 0.5
min_component_value = 0
max_component_value = 255

In [8]:
def get_gradients(image_part):
    grad_n = (abs(image_part[0][1] - image_part[2][1]) +\
              abs(image_part[1][1] - image_part[3][1]) +\
              abs(image_part[0][3] - image_part[2][3]) +\
              abs(image_part[1][3] - image_part[3][3])) / 2 +\
              abs(image_part[0][2] - image_part[2][2]) +\
              abs(image_part[1][2] - image_part[3][2])

    grad_e = (abs(image_part[1][4] - image_part[1][2]) +\
              abs(image_part[1][3] - image_part[1][1]) +\
              abs(image_part[3][4] - image_part[3][2]) +\
              abs(image_part[3][3] - image_part[3][1])) / 2 +\
              abs(image_part[2][4] - image_part[2][2]) +\
              abs(image_part[2][3] - image_part[2][1])

    grad_s = (abs(image_part[4][1] - image_part[2][1]) +\
              abs(image_part[1][1] - image_part[3][1]) +\
              abs(image_part[4][3] - image_part[2][3]) +\
              abs(image_part[1][3] - image_part[3][3])) / 2 +\
              abs(image_part[4][2] - image_part[2][2]) +\
              abs(image_part[1][2] - image_part[3][2])

    grad_w = (abs(image_part[1][0] - image_part[1][2]) +\
              abs(image_part[1][3] - image_part[1][1]) +\
              abs(image_part[3][0] - image_part[3][2]) +\
              abs(image_part[3][3] - image_part[3][1])) / 2 +\
              abs(image_part[2][0] - image_part[2][2]) +\
              abs(image_part[2][3] - image_part[2][1])

    grad_ne = abs(image_part[1][3] - image_part[3][1]) +\
              abs(image_part[0][4] - image_part[2][2]) +\
              abs(image_part[0][3] - image_part[2][1]) +\
              abs(image_part[1][4] - image_part[3][2])

    grad_se = abs(image_part[3][3] - image_part[1][1]) +\
              abs(image_part[4][4] - image_part[2][2]) +\
              abs(image_part[3][4] - image_part[1][2]) +\
              abs(image_part[4][3] - image_part[2][1])

    grad_sw = abs(image_part[4][0] - image_part[2][2]) +\
              abs(image_part[3][1] - image_part[1][3]) +\
              abs(image_part[3][0] - image_part[1][2]) +\
              abs(image_part[4][1] - image_part[2][3])

    grad_nw = abs(image_part[0][0] - image_part[2][2]) +\
              abs(image_part[1][1] - image_part[3][3]) +\
              abs(image_part[1][0] - image_part[3][2]) +\
              abs(image_part[0][1] - image_part[2][3])

    return dict(zip(['grad_n', 'grad_e', 'grad_s', 'grad_w', 'grad_ne', 'grad_se', 'grad_sw', 'grad_nw'],
                    [grad_n, grad_e, grad_s, grad_w, grad_ne, grad_se, grad_sw, grad_nw]))

In [9]:
def get_direction_keys(image_part):
    grad_dict = get_gradients(image_part)
    min_grad, max_grad = min(grad_dict.values()), max(grad_dict.values())
    T = k1 * min_grad + k2 * (min_grad + max_grad)
    grad_dict = {k: v for k, v in grad_dict.items() if v < T}
    return list(get_gradients(image_part).keys())

In [10]:
def count_blue_red_components(image_part):
    keys = get_direction_keys(image_part)

    above_center_comp_sum = 0
    center_comp_sum = 0
    under_center_comp_sum = 0

    if 'grad_n' in keys:
        above_center_comp_sum += image_part[1][2]
        center_comp_sum += (image_part[0][2] + image_part[2][2]) / 2
        under_center_comp_sum += (image_part[0][1] + image_part[0][3] +\
                  image_part[2][1] + image_part[2][3]) / 4

    if 'grad_e' in keys:
        above_center_comp_sum += (image_part[1][2] + image_part[1][4] +\
                  image_part[3][2] + image_part[3][4]) / 4
        center_comp_sum += (image_part[2][4] + image_part[2][2]) / 2
        under_center_comp_sum += image_part[2][3]

    if 'grad_s' in keys:
        above_center_comp_sum += image_part[3][2]
        center_comp_sum += (image_part[4][2] + image_part[2][2]) / 2
        under_center_comp_sum += (image_part[4][1] + image_part[4][3] +\
                  image_part[2][1] + image_part[2][3]) / 4

    if 'grad_w' in keys:
        above_center_comp_sum += (image_part[1][2] + image_part[1][0] +\
                  image_part[3][2] + image_part[3][0]) / 4
        center_comp_sum += (image_part[2][0] + image_part[2][2]) / 2
        under_center_comp_sum += image_part[2][1]

    if 'grad_ne' in keys:
        above_center_comp_sum += (image_part[1][2] + image_part[1][4]) / 2
        center_comp_sum += image_part[1][3]
        under_center_comp_sum += (image_part[0][3] + image_part[2][3]) / 2

    if 'grad_se' in keys:
        above_center_comp_sum += (image_part[3][2] + image_part[3][4]) / 2
        center_comp_sum += image_part[3][3]
        under_center_comp_sum += (image_part[4][3] + image_part[2][3]) / 2

    if 'grad_sw' in keys:
        above_center_comp_sum += (image_part[3][2] + image_part[3][0]) / 2
        center_comp_sum += image_part[3][1]
        under_center_comp_sum += (image_part[4][1] + image_part[2][1]) / 2

    if 'grad_nw' in keys:
        above_center_comp_sum += (image_part[1][0] + image_part[1][2]) / 2
        center_comp_sum += image_part[1][1]
        under_center_comp_sum += (image_part[0][1] + image_part[2][1]) / 2

    above_center_color_comp = image_part[2][2] + (
        above_center_comp_sum - center_comp_sum) / len(keys)
    under_center_color_comp = image_part[2][2] + (
        under_center_comp_sum - center_comp_sum) / len(keys)

    above_clipped = min(max_component_value, max(
        min_component_value, above_center_color_comp))
    under_clipped = min(max_component_value, max(
        min_component_value, under_center_color_comp))

    return above_clipped, under_clipped

In [11]:
def count_color_components(image_part):
    keys = get_direction_keys(image_part)

    green_comp_sum = 0
    center_comp_sum = 0
    side_comp_sum = 0

    if 'grad_n' in keys:
        green_comp_sum += image_part[1][2]
        center_comp_sum += (image_part[0][2] + image_part[2][2]) / 2
        side_comp_sum += (image_part[1][1] + image_part[1][3]) / 2

    if 'grad_e' in keys:
        green_comp_sum += image_part[2][3]
        center_comp_sum += (image_part[2][4] + image_part[2][2]) / 2
        side_comp_sum += (image_part[3][3] + image_part[1][3]) / 2

    if 'grad_s' in keys:
        green_comp_sum += image_part[3][2]
        center_comp_sum += (image_part[4][2] + image_part[2][2]) / 2
        side_comp_sum += (image_part[3][3] + image_part[3][1]) / 2

    if 'grad_w' in keys:
        green_comp_sum += image_part[2][1]
        center_comp_sum += (image_part[2][0] + image_part[2][2]) / 2
        side_comp_sum += (image_part[1][1] + image_part[3][1]) / 2

    if 'grad_ne' in keys:
        green_comp_sum += (image_part[0][3] + image_part[1][2] +\
                           image_part[1][4] + image_part[2][3]) / 4
        center_comp_sum += (image_part[0][4] + image_part[2][2]) / 2
        side_comp_sum += image_part[1][3]

    if 'grad_se' in keys:
        green_comp_sum += (image_part[2][3] + image_part[3][2] +\
                           image_part[3][4] + image_part[4][3]) / 4
        center_comp_sum += (image_part[4][4] + image_part[2][2]) / 2
        side_comp_sum += image_part[3][3]

    if 'grad_sw' in keys:
        green_comp_sum += (image_part[2][1] + image_part[3][0] +\
                           image_part[3][2] + image_part[4][1]) / 4
        center_comp_sum += (image_part[4][0] + image_part[2][2]) / 2
        side_comp_sum += image_part[3][1]

    if 'grad_nw' in keys:
        green_comp_sum += (image_part[0][1] + image_part[1][0] +\
                           image_part[1][2] + image_part[2][1]) / 4
        center_comp_sum += (image_part[0][0] + image_part[2][2]) / 2
        side_comp_sum += image_part[1][1]

    green_comp = image_part[2][2] + (
        green_comp_sum - center_comp_sum) / len(keys)
    side_comp = image_part[2][2] + (
        side_comp_sum - center_comp_sum) / len(keys)

    green_clipped = min(max_component_value, max(
        min_component_value, green_comp))
    side_clipped = min(max_component_value, max(
        min_component_value, side_comp))

    return green_clipped, side_clipped

Примем во внимание, что сетка может располагаться по-разному относительно изображения. Будем ориентироваться на левый верхний пиксель сетки (пиксель, имеющий координаты $(0,0)$).

In [12]:
from enum import Enum

class CornerColor(Enum):
    GREEN_A = 0
    RED = 1
    BLUE = 2
    GREEN_B = 3

In [13]:
def Get_Grid_Corner_Color(image):
    upper_left_grid_corner = None

    if np.argmax(image[0][0]) == 0:
        upper_left_grid_corner = CornerColor.BLUE

    if np.argmax(image[0][0]) == 2:
        upper_left_grid_corner = CornerColor.RED

    if np.argmax(image[0][0]) == 1 and np.argmax(image[0][1]) == 2:
        upper_left_grid_corner = CornerColor.GREEN_A

    if np.argmax(image[0][0]) == 1 and np.argmax(image[0][1]) == 0:
        upper_left_grid_corner = CornerColor.GREEN_B

    return upper_left_grid_corner

Определим, какие индексы (а вернее, их остатки от деления на $2$) будут соответствовать цветам пикселей сетки при различных ее расположениях на изображении.

Например, если в левом верхнем углу сетки фильтра расположен красный цвет (как в тестовом изображении), то индексы строки и столбца для пикселей красного цвета фильтра должны быть одновременно четными (т.е. $(0,0)$, $(0,2)$, $(0,4)$, $(2,0)$, $(2,2)$ и т.д.), а синего - нечетными (т.е. $(1,1)$, $(1,3)$, $(1,5)$, $(3,1)$ и т.д.).

In [14]:
green_a_idxs = [(0, 0, CornerColor.GREEN_A), (0, 1, CornerColor.RED),
                (1, 0, CornerColor.BLUE), (1, 1, CornerColor.GREEN_B)]
red_idxs = [(0, 0, CornerColor.RED), (0, 1, CornerColor.GREEN_A),
            (1, 0, CornerColor.GREEN_B), (1, 1, CornerColor.BLUE)]
blue_idxs = [(0, 0, CornerColor.BLUE), (0, 1, CornerColor.GREEN_B),
            (1, 0, CornerColor.GREEN_A), (1, 1, CornerColor.RED)]
green_b_idxs = [(0, 0, CornerColor.GREEN_B), (0, 1, CornerColor.BLUE),
                (1, 0, CornerColor.RED), (1, 1, CornerColor.GREEN_A)]

Чтобы для любого пикселя изображения можно было выделить окружающую его область размера $5 \times 5$, поместим изображение в рамку толщины $2$, каждый пиксель которой в качестве значения цвета имеет максимальное значение компоненты, соответствующей сетке, продолженной с изображения на рамку.

In [15]:
def VNG_algorithm(image):
    image = image.astype(np.int32)

    upper_left_grid_corner = Get_Grid_Corner_Color(image)

    image = np.max(image, axis=2)

    height = image.shape[0]
    width = image.shape[1]

    blue = np.zeros(image.shape)
    green = np.zeros(image.shape)
    red = np.zeros(image.shape)

    padded_image = np.full((height + 4, width + 4), max_component_value)
    padded_image[2:-2, 2:-2] = image

    for i in range(height):
        for j in range(width):
            image_part = padded_image[i:i+5, j:j+5]
            idxs_with_corner = (i % 2, j % 2, upper_left_grid_corner)

            if idxs_with_corner in green_a_idxs:
                green[i][j] = image[i][j]
                blue[i][j], red[i][j] = count_blue_red_components(image_part)
            elif idxs_with_corner in red_idxs:
                red[i][j] = image[i][j]
                green[i][j], blue[i][j] = count_color_components(image_part)
            elif idxs_with_corner in blue_idxs:
                blue[i][j] = image[i][j]
                green[i][j], red[i][j] = count_color_components(image_part)
            elif idxs_with_corner in green_b_idxs:
                green[i][j] = image[i][j]
                red[i][j], blue[i][j] = count_blue_red_components(image_part)

    final_image = np.transpose(np.array([blue, green, red]), (1, 2, 0)).astype('uint8')

    return final_image

Восстановим тестовое изображение с помощью реализованного алгоритма и запишем его в файл.

In [16]:
%%time
result = VNG_algorithm(rgb_cfa)

CPU times: user 14min 53s, sys: 335 ms, total: 14min 53s
Wall time: 14min 55s


In [17]:
cv2.imwrite('Result.bmp', result)

True

In [18]:
viewImage(result, 'Result.bmp')

Разрешение исходного изображения: $299.875$ пиксели/дюйм.

Разрешение восстановленного изображения: $72.000$ пиксели/дюйм.

In [19]:
ratio = round(299.875 / 72., 3)
print("Разрешение восстановленного изображения по сравнению с оригиналом меньше в {} раз.".format(ratio))

Разрешение восстановленного изображения по сравнению с оригиналом меньше в 4.165 раз.


Вычислим метрику `PSNR` между оригиналом и полученным изображением.

In [20]:
def MSE(orig_img, result):
    coefs = np.array([0.114, 0.587, 0.299])
    orig_y = np.sum(orig_img.astype(np.int32) * coefs, axis=2)
    result_y = np.sum(result.astype(np.int32) * coefs, axis=2)
    mse = np.sum(np.square(orig_y - result_y)) / (result.shape[0] * result.shape[1])
    return mse

In [21]:
def psnr(orig_img, result):
    y_max = 255.
    mse = MSE(orig_img, result)
    psnr = 10. * np.log10(y_max ** 2 / mse)
    return psnr

In [22]:
print("PSNR между оригиналом и полученным изображением = {}.".format(psnr(orig_img, result)))

PSNR между оригиналом и полученным изображением = 26.06088277198723.


Оценим время работы алгоритма в единицах измерения сек/мегапиксель. Время работы в секундах было посчитано выше, оно равно $895$ секундам ($14$ минут $55$ секунд).

In [23]:
time = round(895 / (result.shape[0] * result.shape[1] / 1e6), 3)
print("Время работы алгоритма: {} сек/мегапиксель.".format(time))

Время работы алгоритма: 103.386 сек/мегапиксель.
